## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(['EIN', 'NAME'], axis=1, inplace=True)
print(application_df.head())

  APPLICATION_TYPE       AFFILIATION CLASSIFICATION      USE_CASE  \
0              T10       Independent          C1000    ProductDev   
1               T3       Independent          C2000  Preservation   
2               T5  CompanySponsored          C3000    ProductDev   
3               T3  CompanySponsored          C2000  Preservation   
4               T3       Independent          C1000     Heathcare   

   ORGANIZATION  STATUS     INCOME_AMT SPECIAL_CONSIDERATIONS  ASK_AMT  \
0   Association       1              0                      N     5000   
1  Co-operative       1         1-9999                      N   108590   
2   Association       1              0                      N     5000   
3         Trust       1    10000-24999                      N     6692   
4         Trust       1  100000-499999                      N   142590   

   IS_SUCCESSFUL  
0              1  
1              1  
2              0  
3              1  
4              1  


In [3]:
# Determine the number of unique values in each column.
unique_value_counts = application_df.nunique()
unique_value_counts

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
application_type_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_value = 300
rare_application_types = application_type_counts[application_type_counts < cutoff_value].index.tolist()
application_types_to_replace = rare_application_types

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts_filtered = classification_counts[classification_counts > 1]
classification_counts_filtered

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_value = 500
rare_classification_types = classification_counts[classification_counts < cutoff_value].index.tolist()
classifications_to_replace = rare_classification_types

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
categorical_columns = ['APPLICATION_TYPE', 'CLASSIFICATION', 'AFFILIATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS']
application_df_encoded = pd.get_dummies(application_df, columns=categorical_columns)
application_df_encoded.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [10]:
# Split our preprocessed data into our features and target arrays
X = application_df_encoded.drop('IS_SUCCESSFUL', axis=1)
y = application_df_encoded['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)
# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = X_train.shape[1]
nn_optimized = tf.keras.models.Sequential()
# First hidden layer with dropout
nn_optimized.add(tf.keras.layers.Dense(units=256, activation='relu'))
nn_optimized.add(tf.keras.layers.Dropout(0.3))
# Second hidden layer with dropout
nn_optimized.add(tf.keras.layers.Dense(units=128, activation='relu'))
nn_optimized.add(tf.keras.layers.Dropout(0.3))
# Third hidden layer with dropout
nn_optimized.add(tf.keras.layers.Dense(units=64, activation='relu'))
nn_optimized.add(tf.keras.layers.Dropout(0.3))

# Output layer
nn_optimized.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Build the model
nn_optimized.build(input_shape=(None, input_features))

# Check the structure of the model
nn_optimized.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               11520     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 6

In [13]:
# Compile the model with a lower learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
nn_optimized.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [14]:
# Train the model
fit_model_optimized = nn_optimized.fit(X_train_scaled, y_train, epochs=150, batch_size=256, validation_data=(X_test_scaled, y_test))

Epoch 1/150
101/101 [==============================] - 1s 5ms/step - loss: 0.6062 - accuracy: 0.6976 - val_loss: 0.5617 - val_accuracy: 0.7241
Epoch 2/150
101/101 [==============================] - 0s 4ms/step - loss: 0.5713 - accuracy: 0.7235 - val_loss: 0.5578 - val_accuracy: 0.7273
Epoch 3/150
101/101 [==============================] - 0s 4ms/step - loss: 0.5673 - accuracy: 0.7257 - val_loss: 0.5565 - val_accuracy: 0.7269
Epoch 4/150
101/101 [==============================] - 0s 4ms/step - loss: 0.5615 - accuracy: 0.7272 - val_loss: 0.5587 - val_accuracy: 0.7242
Epoch 5/150
101/101 [==============================] - 0s 4ms/step - loss: 0.5603 - accuracy: 0.7277 - val_loss: 0.5559 - val_accuracy: 0.7292
Epoch 6/150
101/101 [==============================] - 0s 4ms/step - loss: 0.5602 - accuracy: 0.7264 - val_loss: 0.5553 - val_accuracy: 0.7297
Epoch 7/150
101/101 [==============================] - 0s 4ms/step - loss: 0.5578 - accuracy: 0.7285 - val_loss: 0.5527 - val_accuracy: 0.7299

Epoch 58/150
101/101 [==============================] - 0s 3ms/step - loss: 0.5435 - accuracy: 0.7362 - val_loss: 0.5511 - val_accuracy: 0.7325
Epoch 59/150
101/101 [==============================] - 0s 3ms/step - loss: 0.5427 - accuracy: 0.7357 - val_loss: 0.5507 - val_accuracy: 0.7303
Epoch 60/150
101/101 [==============================] - 0s 3ms/step - loss: 0.5438 - accuracy: 0.7352 - val_loss: 0.5523 - val_accuracy: 0.7315
Epoch 61/150
101/101 [==============================] - 0s 4ms/step - loss: 0.5434 - accuracy: 0.7365 - val_loss: 0.5514 - val_accuracy: 0.7299
Epoch 62/150
101/101 [==============================] - 0s 3ms/step - loss: 0.5434 - accuracy: 0.7340 - val_loss: 0.5519 - val_accuracy: 0.7284
Epoch 63/150
101/101 [==============================] - 0s 3ms/step - loss: 0.5433 - accuracy: 0.7348 - val_loss: 0.5517 - val_accuracy: 0.7306
Epoch 64/150
101/101 [==============================] - 0s 3ms/step - loss: 0.5434 - accuracy: 0.7365 - val_loss: 0.5521 - val_accuracy:

Epoch 115/150
101/101 [==============================] - 0s 3ms/step - loss: 0.5391 - accuracy: 0.7379 - val_loss: 0.5551 - val_accuracy: 0.7306
Epoch 116/150
101/101 [==============================] - 0s 3ms/step - loss: 0.5400 - accuracy: 0.7362 - val_loss: 0.5549 - val_accuracy: 0.7305
Epoch 117/150
101/101 [==============================] - 0s 3ms/step - loss: 0.5395 - accuracy: 0.7374 - val_loss: 0.5554 - val_accuracy: 0.7301
Epoch 118/150
101/101 [==============================] - 0s 3ms/step - loss: 0.5397 - accuracy: 0.7374 - val_loss: 0.5553 - val_accuracy: 0.7322
Epoch 119/150
101/101 [==============================] - 0s 3ms/step - loss: 0.5404 - accuracy: 0.7362 - val_loss: 0.5548 - val_accuracy: 0.7314
Epoch 120/150
101/101 [==============================] - 0s 3ms/step - loss: 0.5398 - accuracy: 0.7365 - val_loss: 0.5544 - val_accuracy: 0.7325
Epoch 121/150
101/101 [==============================] - 0s 4ms/step - loss: 0.5391 - accuracy: 0.7365 - val_loss: 0.5537 - val_ac

In [15]:
# Evaluate the model using the test data
model_loss_optimized, model_accuracy_optimized = nn_optimized.evaluate(X_test_scaled, y_test, verbose=2)
model_loss_optimized
model_accuracy_optimized

268/268 - 0s - loss: 0.5578 - accuracy: 0.7314 - 234ms/epoch - 873us/step


0.7314285635948181

In [16]:
# Export our model to HDF5 file
model_filename = "AlphabetSoupCharity_Optimization.h5"
nn_optimized.save("AlphabetSoupCharity_Optimization.h5")

C:\Users\stara\anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
